In [1]:
import pandas as pd
import numpy as np
import os
import random
path = os.getcwd()
name = pd.read_csv(path + '/name_abb.csv')
name_dict = dict(zip(name['Franchise'],name['Abbreviation/Acronym']))

In [2]:
# import the lineup csv
temp = pd.read_csv(path + '/all_combined.csv')

In [3]:
# there is no away team, so I am going to find one.
sche = pd.read_csv(path+'/total_schedule.csv')

sche_home_away_date = sche[['Visitor/Neutral','Home/Neutral',
                            'Home_abb',
                            'Date_fmt',
                            'Season','PTS','PTS.1']]
sche_home_away_date.columns = ['Visitor/Neutral','Home/Neutral',
                               'TEAM','GAMEDAY',
                               'SEASON','AWAYSCORE','HOMESCORE']
# format the date into datetime stamp
sche_home_away_date['GAMEDAY_FMT'] = pd.to_datetime(sche_home_away_date.GAMEDAY)
# find the abbrevation for away team
sche_home_away_date['AWAY'] = [name_dict[n] for n in sche_home_away_date['Visitor/Neutral']]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
# add the season in temp
temp['SEASON'] = [gameday[0:4] for gameday in temp['GAMEDAY']]

In [5]:
sche_home_away_date['SEASON_FMT'] = sche_home_away_date['SEASON'].str.replace( "\-\d\d", "")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
sche_home_away_date.head()

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,GAMEDAY_FMT,AWAY,SEASON_FMT
0,Los Angeles Lakers,Seattle SuperSonics,SEA,2008-01-14,2007-08,123,121,2008-01-14,LAL,2007
1,Houston Rockets,San Antonio Spurs,SAS,2007-11-16,2007-08,84,90,2007-11-16,HOU,2007
2,Sacramento Kings,Boston Celtics,BOS,2007-12-12,2007-08,78,90,2007-12-12,SAC,2007
3,Utah Jazz,Miami Heat,MIA,2007-12-22,2007-08,102,104,2007-12-22,UTA,2007
4,Detroit Pistons,Boston Celtics,BOS,2008-05-28,2007-08,102,106,2008-05-28,DET,2007


# features to calculate

This notebook aims to calculate the advantage part of the featues.
- Offensive Advantages
    - "OFFRTG of A - DEFRTG of B" in the most recent records
        - OFFRTG of A is calculated as the weighted average of the top 10 lineups
        - DEFRTG of B is calculated as the weighted average of the top 10 lineups 

In [ ]:
temp['']

Let us try to start from a small instance to get a feel.

Randomly picked a game.
 - 2018.12.30 MIN vs. MIA
 - Previous n Games

In [7]:
# find all previous games of two team
# including their last season
def findpreviousgames(home, away, pred_day, season, include_last = True):
    prev_season = str(int(season) -1)
    all_last_season_home_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == prev_season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == home) & 
                                                 (sche_home_away_date['AWAY'] == away))]
    all_last_season_away_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == prev_season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == away) & 
                                                 (sche_home_away_date['AWAY'] == home))]
    all_this_season_home_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == home) & 
                                                 (sche_home_away_date['AWAY'] == away))]
    all_this_season_away_games = sche_home_away_date.loc[(sche_home_away_date['SEASON_FMT'] == season) & 
                                                 (sche_home_away_date['GAMEDAY_FMT'] < pred_day) &
                                                 ((sche_home_away_date['TEAM'] == away) & 
                                                 (sche_home_away_date['AWAY'] == home))]
    if include_last:
        return pd.DataFrame(pd.concat([all_last_season_home_games,
                                    all_last_season_away_games,
                                    all_this_season_home_games,
                                    all_this_season_home_games]))
    else:
        return pd.DataFrame(pd.concat([all_this_season_home_games,
                                    all_this_away_games]))

In [8]:
# testing
findpreviousgames(home = 'MIA', away = 'MIN', pred_day = '2018-12-30', season = '2018')

,Visitor/Neutral,Home/Neutral,TEAM,GAMEDAY,SEASON,AWAYSCORE,HOMESCORE,GAMEDAY_FMT,AWAY,SEASON_FMT
13971,Minnesota Timberwolves,Miami Heat,MIA,2017-10-30,2017-18,125,122,2017-10-30,MIN,2017
13924,Miami Heat,Minnesota Timberwolves,MIN,2017-11-24,2017-18,109,97,2017-11-24,MIA,2017


In [ ]:
# find lineups data
def find_lineups_stats(team,gameday):
    stats = temp.loc[(temp['TEAM'] == team) & (temp['GAMEDAY'] == gameday)]
    return stats

# calculate the relevant features: weighted average of their feature
# weighted by time played in the game
def calculate_weighted_feature(season,lineup_stats,
                               feature,
                               max_lineup_considered = 10, 
                               time_disc = 0.90):
    try:
        total_time = np.sum(lineup_stats['MIN'])
        weighted_feature = 0

        for index, lineup in lineup_stats.iterrows():

            # previous data is discounted
            if int(lineup['SEASON']) < int(season):
                weighted_feature += \
                time_disc * (((lineup['MIN']/total_time) * lineup[feature]))
            else:
                weighted_feature += \
                (((lineup['MIN']/total_time) * lineup[feature]))
            if index > 10:
                break
        
        return weighted_feature
            
    except:
        print('Error')
        print(lineup_stats.head())
    

In [ ]:
# testing:
bos181016 = find_lineups_stats('BOS','2018-10-16')
# print(calculate_weighted_feature(lineup_stats=bos181016,feature = 'OFFRTG'))
# print(calculate_weighted_feature(lineup_stats=bos181016,feature = 'DEFRTG'))

In [ ]:
# calculate the feature for game 2018.12.30 MIN vs. MIA
previous_games_test = findpreviousgames(home = 'MIA', away = 'MIN', pred_day = '2018-12-30', season = '2018')
# find the relevant lineup stats for two teams
def find_lineup_records_for_two_teams(games):
    team1_stats = pd.DataFrame()
    team2_stats = pd.DataFrame()
    for index, game in games.iterrows():
        # we don't know who is home
        team1 = game['TEAM']
        team2 = game['AWAY']
        if index == 1:
            team1_stats = find_lineups_stats(team1,game['GAMEDAY'])
            team2_stats = find_lineups_stats(team2,game['GAMEDAY'])
        else:
            team1_stats = pd.concat([team1_stats,find_lineups_stats(team1,game['GAMEDAY'])])
            team2_stats = pd.concat([team2_stats,find_lineups_stats(team2,game['GAMEDAY'])])
    
    return (team1_stats,team2_stats)

In [ ]:
# testing
result0 = find_lineup_records_for_two_teams(previous_games_test)[0]
result1 = find_lineup_records_for_two_teams(previous_games_test)[1]
print('Weighted OFF of MIA:', calculate_weighted_feature(pred_season = '2018', lineup_stats=result0,feature = 'OFFRTG'))
print('Weighted OFF of MIN:', calculate_weighted_feature(pred_season = '2018', lineup_stats=result1,feature = 'OFFRTG'))

In [ ]:
previous_games_test = findpreviousgames(home = 'MIA', away = 'MIN', pred_day = '2018-12-30', season = '2018')
previous_games_test

# Next Step:
- calculate the feature for all games records
- The final table is called 'targ_data'

In [ ]:
targ_data = sche_home_away_date[['TEAM','AWAY',
                                'GAMEDAY_FMT','SEASON_FMT',
                                'AWAYSCORE','HOMESCORE']]
# construct a target variable
targ_data['HOMEWIN'] = [int(int(game['AWAYSCORE']) < int(game['HOMESCORE'])) \
                        for index, game in targ_data.iterrows()]

For each game, we should:
- Find all its previous games
- Calculate the weighted average of the features
    - nextstep is to make the calculation of a list of features possible

In [ ]:
# testing on five games
targ_data_test = targ_data.iloc[255:260]
HOME_FEATURE1,AWAY_FEATURE1,HOME_FEATURE2,AWAY_FEATURE2 = ([] for _ in range(4))
homestorage = dict(zip(list(range(1,3)),[HOME_FEATURE1,HOME_FEATURE2]))
awaystorage = dict(zip(list(range(1,3)),[AWAY_FEATURE1,AWAY_FEATURE2]))
for index, game in targ_data_test.iterrows():

    # first finds all previous games
    home = game['TEAM']
    away = game['AWAY']
    gameday = game['GAMEDAY_FMT']
    season = game['SEASON_FMT']
    feature = ['OFFRTG','DEFRTG']
    home_feature = 0
    away_feature = 0
    previous_games_now = findpreviousgames(home = home, away = away,
                                      pred_day = gameday,
                                      season = season)
    
    # then finds the relevant lineup stats
    homestats, awaystats = find_lineup_records_for_two_teams(previous_games_now)
    
    # then calculate the weighted features
    for i in range(len(feature)):
        home_feature = calculate_weighted_feature(season = season, 
                                                   lineup_stats= homestats,
                                                   feature = feature[i])

        away_feature = calculate_weighted_feature(season = season, 
                                                   lineup_stats= awaystats,
                                                   feature = feature[i])
        # store it somewhere
        homestorage[i+1].append(home_feature)
        awaystorage[i+1].append(away_feature)


In [ ]:
targ_data_test

In [ ]:
# findpreviousgames(home = 'SEA', away = 'CHA', pred_day = '2008-03-28', season = '2007')

In [ ]:
storage = dict(zip(list(range(1,5)),[HOME_FEATURE1,AWAY_FEATURE1,HOME_FEATURE2,AWAY_FEATURE2]))

In [ ]:
homestorage

In [ ]:
awaystorage